# Chapter 3 - notebook level 1

## Save NLIW event data in ENU coordinates

In [1]:
import os
import glob
import numpy as np
import xarray as xr
import natsort

# from d2spike.despike import DataArray
# from wootils.plotnice import vert_stack
from pIMOS.utils.nortek_signature_utils import beam2inst, inst2earth

import sys
sys.path.insert(0, '../../')
from chp3_despike import full_pipe
from chp3_basefuncs import get_sigfiles
from chp3_fluxfuncs import  get_nliw_times

import warnings
warnings.filterwarnings("ignore")

# sudo mount -t drvfs '\\drive.irds.uwa.edu.au\OGS-ODFDA-001' /mnt/share

pyODAS not found, not importing pIMOS.xrwrap.rsi_vmp


In [2]:
moor = 'L150'
field_trip = 'RS19'

## Specify the events

In [3]:
wav_tx = get_nliw_times()

# Set key data directories
nc_dir = r'/mnt/share/Experiments/rs19/archive_FV00/Signature 1000/'
nc_files = glob.glob(os.path.join(nc_dir, f'*{moor}*Signature*.nc'))

# Files need natural sorting!!!
nc_files = natsort.natsorted(nc_files)
# nc_files

t_bef = np.timedelta64(30,'m')
t_aft = np.timedelta64(180,'m')

sig_fil = get_sigfiles(wav_tx, t_bef, t_aft, nc_files)

In [4]:
# Set some parameters
gf_sig = [2,2]
re1 = 0.05
re2 = 0.01
max_z = 0.07
max_t = 600 # milliseconds

In [5]:
save_dir = r'/mnt/c/Users/00099894/Large_Data/RS2019'
save_dir = os.path.join(save_dir, 'NLIW')

In [6]:
sig_fil[0]

'[RS2019]_[RS19_REC]_[L150]_[28]_[Signature 1000]_[100460]_[13of59].nc'

In [7]:
sig_full = os.path.join(nc_dir, sig_fil[0])
ds1 = xr.open_dataset(sig_full)

In [8]:
ds1

<xarray.Dataset>
Dimensions:                   (beam: 5, height: 254, time: 204479,
                               cartesian_axes: 3)
Coordinates:
  * beam                      (beam) int32 1 2 3 4 5
  * cartesian_axes            (cartesian_axes) int32 1 2 3
  * height                    (height) float64 0.1 0.13 0.16 ... 7.63 7.66 7.69
  * time                      (time) datetime64[ns] 2019-03-12T16:35:59.06340...
    lat_nom                   float64 ...
    lon_nom                   float64 ...
    z_nom                     float64 ...
Data variables: (12/21)
    vel_dolfyn                (beam, height, time) float32 ...
    vel_enu                   (cartesian_axes, height, time) float32 ...
    vel_xyz                   (cartesian_axes, height, time) float32 ...
    four_beam_error_velocity  (height, time) float64 ...
    pressure                  (time) float32 ...
    temperature               (time) float32 ...
    ...                        ...
    qc_velocity               (beam, height, time) float32 ...
    qc_velocity3              (cartesian_axes, height, time) float64 ...
    qc_temperature            (time) float32 ...
    qc_pressure               (time) float32 ...
    qc_compass                (time) float32 ...
    qc_tilt                   (time) float32 ...
Attributes: (12/33)
    title:                           Measured data from a Nortek Signature
    institution:                     The University of Western Australia
    institution_division:            Ocean Dynamics
    source:                          Nortek Signature ADCP
    project:                         RS2019
    history:                         
    ...                              ...
    nominal_site_depth:              150.0
    pressure_sensor_height_asb:      
    nominal_instrument_height_asb:   0.29
    nominal_instrument_orientation:  
    timezone:                        UTC
    is_profile_data:                 0

In [9]:
# Loop through files
for ixll, (file, wave) in enumerate(zip(sig_fil, wav_tx)):
    print(ixll)
    if ixll>6:
    
        # Load Signature file
        if len(file) == 2:
            sig_full = os.path.join(nc_dir, file[0])
            ds1 = xr.open_dataset(sig_full)
            s_ix1 = (ds1['time'].values >= wave - t_bef) &\
                    (ds1['time'].values <= wave + t_aft)
            xr.Dataset.close(ds1)

            sig_full = os.path.join(nc_dir, file[1])
            ds2 = xr.open_dataset(sig_full)
            s_ix2 = (ds2['time'].values >= wave - t_bef) &\
                    (ds2['time'].values <= wave + t_aft)
            xr.Dataset.close(ds2)
            
            if (len(ds2.time.isel(time=s_ix2)) > 0):
                print('Concat')
                ds = xr.concat([ds1.isel(time=s_ix1), ds2.isel(time=s_ix2)], dim='time')
                
                del ds1, ds2
            else:
                ds = ds1 
        else:
            sig_full = os.path.join(nc_dir, file)
            ds = xr.open_dataset(sig_full)
            xr.Dataset.close(ds)

        ## Extract the event
        # s_ix = (ds['time'].values >= st) &\
        #         (ds['time'].values <= ed)
        s_ix = (ds['time'].values >= wave - t_bef) &\
                    (ds['time'].values <= wave + t_aft)

        # Copy out the new data array to fill
        w_new = ds['vel_dolfyn'].sel(time=s_ix).copy()

        for b in ds.beam.values:
            w = ds['vel_dolfyn'].sel(beam=b, time=s_ix)
            c = ds['corr'].sel(beam=b, time=s_ix)
            w_new[b-1] = full_pipe(w.T, c.T, corrflag=45, qc0_val=0.5, gf_sig=gf_sig, re1=re1, re2=re2).T

            # if b==5:
            #     fig, ax = vert_stack(1, hsize=14, vsize=4)
            #     w_new.sel(beam=b)[::4,::32].plot(cmap='PuOr',\
            #                                     vmin=-np.abs(w_new.sel(beam=b)).max(),\
            #                                     vmax=np.abs(w_new.sel(beam=b)).max(),\
            #                                     ax=ax, center=0, cbar_kwargs={'pad':0.01})
        
        # Run the 2D interpolation
        w_intCT = w_new.values.copy()
        for ix, b in enumerate(ds.beam.values):
            w_b = w_new.sel(beam=b).T
            if np.sum(np.isnan(w_b.values)) > 0:
                print('Processing beam ' + str(b))
                w_intCT[ix] = w_b.floatda.interp_2D().T
            
        
        w_intCT = xr.DataArray(data=w_intCT, coords=w_new.coords)
        # w_intCT = w_intCT.interpolate_na(dim='height', method='cubic', max_gap=max_z)
        # w_intCT = w_intCT.interpolate_na(dim='time', method='cubic', max_gap=np.timedelta64(max_t,'ms'))
        
        # fig, ax = vert_stack(1, hsize=14, vsize=4)
        # w_intCT.sel(beam=5)[::4,::32].plot(cmap='PuOr',\
        #                                     vmin=-np.abs(w_intCT.sel(beam=5)).max(),\
        #                                     vmax=np.abs(w_intCT.sel(beam=5)).max(),\
        #                                     ax=ax, center=0, cbar_kwargs={'pad':0.01})

        # Save the data files
        ds_new = ds[['heading', 'pitch', 'roll']].isel(time=s_ix)
        ds_new['beam_vel'] = xr.DataArray(data=w_intCT.values, coords=w_intCT.coords)

        # # Save as netcdf
        # ds_new.to_netcdf(path=('NLIW_' + str(ixll+1) + '_Beam.nc'))

        # Convert to XYZ
        T = eval(ds.attrs['raw_file_attributes'])['config:TransMatrix']
        T = T.replace('[','');T=T.replace(']','');T=np.fromstring(T, dtype=float, sep=' ').reshape((4, 4))

        beam_vel = w_intCT.isel(beam=np.arange(4)).values
        XYZZ = beam2inst(beam_vel, T)[0:3, :, :]
        w_instr = xr.DataArray(data=XYZZ, coords={'cartesian_axes': ds.cartesian_axes,\
                                                'height': w_intCT.height,\
                                                'time': w_intCT.time})

        # Convert to ENU
        ENU = inst2earth(w_instr.values,\
                        ds['heading'].sel(time=s_ix),\
                        ds['pitch'].sel(time=s_ix),\
                        ds['roll'].sel(time=s_ix),\
                        orientation='up',\
                        fixed_orientation=True)
        uvw_earth = xr.DataArray(data=ENU, coords={'cartesian_axes': ds.cartesian_axes,\
                                                    'height': w_instr.height,\
                                                    'time': w_instr.time})    

        ds_earth = ds[['heading', 'pitch', 'roll']].isel(time=s_ix)
        ds_earth['enu'] = xr.DataArray(data=uvw_earth.values, coords=uvw_earth.coords)
        ds_earth['beam5'] = ds_new['beam_vel'].sel(beam=5)
        ds_earth['echo'] = ds['echo'].isel(time=s_ix, beam=4)

        # Save as netcdf
        ds_earth.to_netcdf(path=os.path.join(save_dir, 'NLIW_' + str(ixll+1) + '_ENU.nc'))
        del ds, ds_new, ds_earth, w_new, w_intCT, w_instr, uvw_earth

0
1
2
3
4
5
6
7
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
Processing beam 1
Processing beam 2
Processing beam 3
Processing beam 4
Processing beam 5
8
Concat
(50399,)
(50399,)
(50399,)
(50399,)
(50399,)
(50399,)
(50399,)
(50399,)
(50399,)
(50399,)
Processing beam 1
Processing beam 2
Processing beam 3
Processing beam 4
Processing beam 5
9
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
Processing beam 1
Processing beam 2
Processing beam 3
Processing beam 4
Processing beam 5
10
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
(50400,)
Processing beam 1
Processing beam 2
Processing beam 3
Processing beam 4
Processing beam 5
